In [1]:
import matplotlib.pyplot as plt
import sys
import os
# Add the parent directory to the path to import src as a package
sys.path.insert(0, os.path.abspath('..'))
from src import dataloader
import importlib
importlib.reload(dataloader)
import numpy as np

%matplotlib widget
plot_flag = False


# Test of synchronization between the ET and EEG data

- Experiments were conducted for dyads  "W003" "W020" "W030" "W036" 
- For alignment test we use the comparison of blinks detecte by ET and the signal from Fp1 EEG channel. 
- To have a smooth EEG signal filter it bandpass between 1 and 10 Hz.

**The estimated delay is ~0.3s** which needs to be subtracted from time axis of ET signals. This operation is implemented in `dataloader.py` module, function `load_et_data`

In [2]:
import ipywidgets as widgets
from IPython.display import display

# Create dropdown widget for dyad selection
dyad_selector = widgets.Dropdown(
    options=['W003', 'W020', 'W030', 'W036'],
    value='W003',
    description='Dyad ID:',
    disabled=False,
)

display(dyad_selector)

Dropdown(description='Dyad ID:', options=('W003', 'W020', 'W030', 'W036'), value='W003')

In [3]:
# creates a DataLoader class object that creates a structure described in the docs folder (data_structure_spec.md) from raw data

dyad_id = dyad_selector.value
lowcut=1.0
highcut=10.0
eeg_filter_type = 'iir' # choose 'fir' or 'iir' for EEG filtering
multimodal_data = dataloader.create_multimodal_data(data_base_path = "../data", 
                                                    dyad_id = dyad_id, 
                                                    load_eeg=True, 
                                                    load_et=True, 
                                                    lowcut=lowcut, 
                                                    highcut=highcut, 
                                                    eeg_filter_type=eeg_filter_type, 
                                                    interpolate_et_during_blinks_threshold=0.3,
                                                    median_filter_size=64,
                                                    low_pass_et_order=351,
                                                    et_pos_cutoff=128,
                                                    et_pupil_cutoff=4,
                                                    pupil_model_confidence=0.9,
                                                    plot_flag=plot_flag)

Detected events: [{'name': 'Brave', 'start': 382.1572265625, 'duration': 59.3291015625}, {'name': 'Peppa', 'start': 312.2607421875, 'duration': 59.61328125}, {'name': 'Incredibles', 'start': 242.8125, 'duration': 59.21484375}, {'name': 'Talk_1'}, {'name': 'Talk_2'}]
Applying iir filters to EEG data.
Reseting the EEG time to the start of Incredibles
ET time range: 236.36s to 1095.13s
Events from ET annotations:
[None 'Incredibles' 'Peppa' 'Brave']
Events from ET annotations:
[None 'Incredibles' 'Peppa' 'Brave' 'Talk1']
Reseting the ET time to the start of Incredibles
Processing member: ch, blink column: ET_ch_blinks
Processing member: cg, blink column: ET_cg_blinks
Event Incredibles start times are consistent within 0.0 seconds.
Event Peppa differ in start times by: abs(0.02734375) seconds.
Event Brave differ in start times by: abs(0.0078125) seconds.
Events column created based on EEG_events and ET_event columns.
Event structure created based on events column.


UnboundLocalError: cannot access local variable 'print_events' where it is not associated with a value

In [ ]:
multimodal_data.events

In [ ]:
multimodal_data.data.columns

## Test the aligment of the EEG and ET events

In [ ]:
event_mapping_ET = {event: idx for idx, event in enumerate(multimodal_data.data['ET_event'].unique())}
print("Event mapping:", event_mapping_ET)

# Map ET_event to integer values
multimodal_data.data['event_num_ET'] = multimodal_data.data['ET_event'].map(event_mapping_ET)
# create a mapping from unique events values to integers
event_mapping_EEG = {event: idx for idx, event in enumerate(multimodal_data.data['EEG_events'].unique())}
print("Event mapping for events:", event_mapping_EEG)
multimodal_data.data['event_num_EEG'] = multimodal_data.data['EEG_events'].map(event_mapping_EEG)

# create mapping from events to integer values
event_mapping_all = {event: idx for idx, event in enumerate(multimodal_data.data['events'].unique())}
multimodal_data.data['event_num_all'] = multimodal_data.data['events'].map(event_mapping_all)
print("Event mapping for all events:", event_mapping_all)

plt.figure(figsize=(15, 6))
plt.plot(multimodal_data.data['time'], multimodal_data.data['diode'], label='diode')
plt.plot(multimodal_data.data['time'], multimodal_data.data['event_num_ET'],'o', label='ET events')
plt.plot(multimodal_data.data['time'], multimodal_data.data['event_num_EEG'],'+', label='EEG Events')
plt.plot(multimodal_data.data['time'], multimodal_data.data['event_num_all'],'x', label='All Events')
plt.xlabel('Time (s)')
plt.legend()
#plt.xlim(-10,200)
plt.show()

## Test the aligment of blinks

Select caregiver or child data and zoom around blink markers to make sure the `delta` is OK. 

In [ ]:
delta = 0.0  # to align ET with EEG (seconds): after correction applied in load_et_data the delta here should be 0 
who = 'ch' # choose 'cg' or 'ch' for caregiver or child data
data_selected = multimodal_data.data[(multimodal_data.data.time>-10)&(multimodal_data.data.time<200)]

plt.figure(figsize=(15, 6))
plt.plot(data_selected['time'], data_selected['diode'], linestyle='-', label='diode')
plt.plot(data_selected['time'], (data_selected[f'EEG_{who}_Fp1'])/np.max(data_selected[f'EEG_{who}_Fp1']), linestyle='-', label=f'EEG_{who}_Fp1')
plt.plot(data_selected['time']-delta, data_selected[f'ET_{who}_blinks'], linestyle='-', label=f'ET_{who}_blinks')
plt.legend()
plt.title(f'Dyad {dyad_id} - EEG and ET synchronization test for {who}')
plt.xlabel('Time')
plt.ylim(-2, 2)
plt.show()

## Test if blinks are aligned with x-coordinate  and pupli diameter?
Zoom around blinks and chek if the jumps in ET_ch_x and pupil allign with blinks

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(multimodal_data.data['time'], multimodal_data.data[f'ET_{who}_x'], label=f'ET_{who}_x')
plt.plot(multimodal_data.data['time'], multimodal_data.data[f'ET_{who}_blinks'], label=f'ET_{who}_blinks')
plt.plot(multimodal_data.data['time'], multimodal_data.data[f'ET_{who}_pupil']-np.mean(multimodal_data.data[f'ET_{who}_pupil']), label=f'ET_{who}_pupil')
plt.legend()
plt.xlim(-10,200)
plt.show()

## Check the relation of IBI and pupil diameter

In [ ]:
from scipy import stats
movie = 'Incredibles'
data_selected = multimodal_data.data[(multimodal_data.data.events == movie)]
plt.figure(figsize=(15, 6))
plt.plot(data_selected['time'], (data_selected[f'IBI_{who}'] - data_selected[f'IBI_{who}'].mean()) / data_selected[f'IBI_{who}'].std(), label=f'IBI_{who}')
plt.plot(data_selected['time'], (data_selected[f'ET_{who}_pupil'] - data_selected[f'ET_{who}_pupil'].mean()) / data_selected[f'ET_{who}_pupil'].std(), label=f'ET_{who}_pupil')
plt.legend()
plt.title(f'Dyad {dyad_id} - IBI and ET pupil diameter synchronization test for {who} during {movie}')
plt.show()